In [832]:
#Import packages
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import missingno as msno 
from scipy.stats import norm, skew, stats
import statsmodels.api as sm
from sklearn.metrics import r2_score


In [1046]:
train = pd.read_csv('train_processed.csv')
test = pd.read_csv('test_processed.csv')
pd.set_option('display.max_columns', None)

In [1047]:
y_train = train['SalePrice']
train.drop('SalePrice', axis=1, inplace=True)
print(y_train.shape)
test['SalePrice'] = 0
y_test = test['SalePrice'] 
AmesHousing_totalDF = pd.concat([train,test], ignore_index=True)
AmesHousing_totalDF.drop('SalePrice', axis=1, inplace=True)

AmesHousing_totalDF.shape


(1458,)


(2917, 64)

In [1048]:
def dummify_catFeat(df):
    dummy = pd.get_dummies(df, drop_first=True)
    del_feat = df.select_dtypes(['object']).columns.to_list()
    for feature in del_feat:
        df.drop(feature, axis=1, inplace=True)
    df = pd.concat([df, dummy], axis=1)
    return df

AmesHousing_dummies = dummify_catFeat(AmesHousing_totalDF)
print(AmesHousing_dummies.shape)
# AmesHousing_dummies.head()
test_dummies = AmesHousing_dummies[1460:].copy()

(2917, 234)


In [1049]:
# AmesHousing_dummies.fillna(0, inplace=True)
# np.sum(np.isnan(AmesHousing_dummies)).max()

In [1050]:
try:  # train_test_split was moved in 0.18.0
    from sklearn.model_selection import train_test_split
except:  # Following import works through 0.19 but outputs a warning in 0.18
    from sklearn.cross_validation import train_test_split
x_train, x_test, y_train, y_test = train_test_split(AmesHousing_dummies[:1458], y_train, test_size=0.3, random_state=42)
# x_train, x_test, y_train, y_test = train_test_split(train_dummies, y_train, test_size=0.3, random_state=42)

# x_train = AmesHousing_dummies[:1460].copy()
# x_test = AmesHousing_dummies[1460:].copy()

print('x_train:', x_train.shape)
print('x_test:', x_test.shape)
print('y_train:', y_train.shape)
print('y_test', y_test.shape)

x_train: (1020, 234)
x_test: (438, 234)
y_train: (1020,)
y_test (438,)


In [1051]:
from sklearn.linear_model import LinearRegression
ols = LinearRegression()
ols.fit(x_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [1052]:
print('R^2 is equal to %.3f' %(ols.score(x_train, y_train)))
print('RSS is equal to %.3f' %(np.sum((y_train-ols.predict(x_train))**2)))
print('The intercept is %.3f' %(ols.intercept_))
# print('The slopes are %s' %(ols.coef_))


R^2 is equal to 0.951
RSS is equal to 8.072
The intercept is 4.109


In [1053]:
r2_score(y_train, ols.predict(x_train))
# ols.predict(x_train)

0.9507611593610075

In [1054]:
print(ols.score(x_test, y_test))
print(r2_score(y_test, ols.predict(x_test)))

0.8941289183551646
0.8941289183551646


In [1043]:
import statsmodels.api as sm 
X_add_const = sm.add_constant(x_train) 
ols = sm.OLS(y_train, X_add_const)
results = ols.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:              SalePrice   R-squared:                       0.951
Model:                            OLS   Adj. R-squared:                  0.940
Method:                 Least Squares   F-statistic:                     84.80
Date:                Sun, 20 Dec 2020   Prob (F-statistic):               0.00
Time:                        17:01:59   Log-Likelihood:                 1020.7
No. Observations:                1020   AIC:                            -1661.
Df Residuals:                     830   BIC:                            -725.1
Df Model:                         189                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                     4.10

/opt/anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:1362: RuntimeWarning: invalid value encountered in true_divide
  return self.params / self.bse
/opt/anaconda3/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
/opt/anaconda3/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
/opt/anaconda3/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:1912: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)


In [1044]:
pValues = results.pvalues
print('The significant coefficients')
print(pValues[pValues<0.05])

The significant coefficients
const                    1.278070e-05
LotFrontage              1.523981e-02
LotArea                  2.029733e-04
OverallQual              3.271548e-10
OverallCond              1.381489e-14
                             ...     
Functional_Typ           2.514729e-02
SaleType_New             2.381098e-06
SaleCondition_Alloca     2.983109e-02
SaleCondition_Normal     4.898234e-06
SaleCondition_Partial    1.015536e-04
Length: 64, dtype: float64


In [1032]:
# Using log(SalePrice)
# y_trainLog = np.log(y_train)
# ols2 = LinearRegression()
# ols2.fit(x_train, y_trainLog)
# print('R^2 is equal to %.3f' %(ols2.score(x_train, y_trainLog)))


R^2 is equal to 0.951


In [1033]:
# r2_score(np.log(y_test), ols2.predict(x_test))

0.8941289183551643

In [1030]:
# ols.predict(test_dummies)